In [105]:
import torch
import torch.nn as nn
from sklearn import datasets
import numpy as np

In [106]:
df = datasets.load_breast_cancer()

In [107]:
x = df.data
y = df.target
print(x.shape, y.shape)
x

(569, 30) (569,)


array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

In [109]:
print(X_train.shape, y_train.shape)

(341, 30) (341,)


In [110]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [111]:
X_train

array([[-0.93158266, -1.58538382, -0.96082569, ..., -1.29685162,
        -1.15903886, -0.96557241],
       [-0.55984676,  0.73956333, -0.59548627, ..., -0.80908172,
         0.35905959, -0.08051317],
       [-0.42113933, -0.58305279, -0.42049516, ..., -0.54353364,
         0.3130566 ,  0.12032719],
       ...,
       [ 0.00607953, -0.55540228, -0.10567281, ..., -1.27526614,
        -0.65935128, -1.2622942 ],
       [-0.07991907,  0.08977632, -0.07253362, ...,  1.04431968,
         0.48437806,  1.25104711],
       [-0.5792658 ,  0.29715515, -0.63428338, ..., -0.63888248,
        -0.28498218, -0.83224618]])

In [112]:
X_test

array([[-0.49881549, -0.14525303, -0.47788255, ..., -0.2166455 ,
         0.22263695,  0.22982491],
       [ 1.29605858,  0.47918768,  1.2401018 , ...,  0.985309  ,
        -0.51182448, -0.99620908],
       [ 0.33065491,  0.05751739,  0.35504282, ...,  0.55825798,
        -0.05813988, -0.18490477],
       ...,
       [-1.33244711,  0.41006141, -1.32576096, ..., -0.78081871,
         0.00689882,  0.49137126],
       [ 2.07559431,  0.40545299,  2.19790535, ...,  1.06606046,
        -0.06448512,  0.29109824],
       [-0.30739925,  0.30176357, -0.357854  , ..., -1.14140506,
        -0.29608635, -0.51963872]])

In [113]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [114]:
print(y_train.shape)

torch.Size([341, 1])


In [115]:
class Logistic_reg(nn.Module):
    def __init__(self, input_feat):
        super(Logistic_reg, self).__init__()
        self.l1 = nn.Linear(input_feat, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.l1(x))
        return y_pred

In [116]:
model = Logistic_reg(n_feat)

In [117]:
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

itr = 100

In [125]:
for epoch in range(itr):

    y_pred = model(X_train)

    loss_val = loss(y_pred, y_train)

    loss_val.backward()

    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss: {loss_val.item()}')

epoch: 10, loss: 0.0962638184428215
epoch: 20, loss: 0.09593716263771057
epoch: 30, loss: 0.09561701864004135
epoch: 40, loss: 0.09530316293239594
epoch: 50, loss: 0.09499535709619522
epoch: 60, loss: 0.09469343721866608
epoch: 70, loss: 0.09439720213413239
epoch: 80, loss: 0.09410643577575684
epoch: 90, loss: 0.09382101148366928
epoch: 100, loss: 0.0935407280921936


In [126]:
predicted = model(X_test)
predicted = predicted.round()

acc = predicted.eq(y_test).sum() / float(y_test.shape[0])

In [127]:
print(f'accuracy is {acc}')

accuracy is 0.9868420958518982
